In [120]:
from pyscf.lib import krylov
from pyscf import gto, scf
import scipy
import inspect
import pyscf.qmmm
import pyscf.dft
import numpy as np
angstrom = 1 / 0.52917721067
from matplotlib import pyplot as plt
from functools import reduce
from numpy.linalg import inv 
from pyscf.scf import cphf 

def fc(calc,deltaZ):
    mf = pyscf.qmmm.mm_charge(calc, calc.mol.atom_coords(), deltaZ)  # now is add_mm_charge
    class NoSelfQMMM(mf.__class__):
        def energy_nuc(self):
            q = self.mol.atom_charges().astype(np.float).copy()
            q1 =q+ np.asarray(deltaZ) 
            return self.mol.energy_nuc(q1)
    return(NoSelfQMMM(mf,mf.mm_mol))

def DeltaV(mol,dL):
    mol.set_rinv_orig_(mol.atom_coords()[0])
    dV=mol.intor('int1e_rinv')*dL[0]
    mol.set_rinv_orig_(mol.atom_coords()[1])
    dV+=mol.intor('int1e_rinv')*dL[1]
    return -dV

In [121]:
mol=gto.Mole(atom="C 0 0 0 ;O 0 0 2.", unit="Bohr", basis= "sto-3g")
mf=scf.RHF(mol)
mf.scf()
dL=.001
mf1=fc(mf,[dL,-dL])
mf1.scf()
mf2=fc(mf,[-dL,dL])
mf2.scf()
dV=DeltaV(mol,[dL,-dL])
h2=mf.get_veff()
dh2=(mf1.get_veff()-mf2.get_veff())/2
J,K=mf.get_jk()
g_ijkl=mf.mol.intor('int2e_sph')
g_h2=g_ijkl-np.swapaxes(g_ijkl,1,2)/2
nocc= mol.nelec[0]


C=mf.mo_coeff
nmo=C.shape[0]
nvir=nmo-nocc
S=mf.get_ovlp()
e=mf.mo_energy
E=np.diag(e)
O=np.diag(mf.mo_occ)
C1=mf1.mo_coeff
P=mf.make_rdm1()
P1=mf1.make_rdm1()
P2=mf2.make_rdm1() 
dP=(P1-P2)/2
dP2=(P1-2*P+P2)/dL**2
dC=(abs(C1)*C/abs(C)-C)
dV_mo=C.T@dV@C

converged SCF energy = -111.199724042754
converged SCF energy = -111.192442208578


Initialize <pyscf.gto.mole.Mole object at 0x7f972f3bbda0> in <pyscf.scf.hf.RHF object at 0x7f972f3bbef0>


converged SCF energy = -111.207008805405


In [122]:
def test(u_): # check density matrices comparing to the one ontained by FD
    ut=np.zeros((nmo,nmo))
    ut[nocc:,:nocc]=u_.reshape(nvir,nocc)
    ut[:nocc,nocc:]=-u_.reshape(nvir,nocc).T
    dP_app=C@(ut@O-O@ut)@C.T    
    print(np.linalg.norm(dP))
    print(np.linalg.norm(dP_app))
    print(np.linalg.norm(dP_app-dP))

In [123]:
nmo,nocc,nvir

(10, 7, 3)

In [124]:
em1,em2=np.meshgrid(e,e)
emeshvo=(em1-em2)[nocc:,:nocc]
emeshf=emeshvo.flatten()
G_cphf=g_ijkl.copy()
G_cphf=4*G_cphf -G_cphf.swapaxes(3,1)-G_cphf.swapaxes(1,2)
G_cphf=G_cphf@C
G_cphf=(G_cphf.swapaxes(2,3)@C).swapaxes(3,2)
G_cphf=(G_cphf.swapaxes(1,3)@C).swapaxes(3,1)
G_cphf=(G_cphf.swapaxes(0,3)@C).swapaxes(3,0)

G_cphfvovo=G_cphf.copy()[nocc:,:nocc,nocc:,:nocc]    #aibj

In [125]:
U_0=dV_mo.copy()
for i in range(mol.nao):
    for a in range(mol.nao):
        if e[i] !=e[a] :
            U_0[i,a]/=(e[a]-e[i])

In [126]:
U_0vo=U_0[nocc:,:nocc]
U_0f=U_0vo.flatten()

In [127]:
dVvo=(dV_mo[nocc:,:nocc])
dVf=dVvo.flatten()

In [128]:
print(dVvo/emeshvo,'\n')
print(U_0vo)

[[ 6.09412400e-21  1.64229498e-20 -1.93372085e-19 -9.05402235e-19
   5.52847144e-04 -8.45971617e-06 -7.04013080e-19]
 [-1.32721528e-20  4.44527460e-21  5.38668155e-21  2.04544456e-19
   8.45971617e-06  5.52847144e-04  1.04041849e-19]
 [-2.60636670e-05 -2.90750246e-05  1.92854913e-04 -1.26016070e-04
  -2.46495168e-19  5.08792645e-20 -1.64871398e-04]] 

[[ 6.09412400e-21  1.64229498e-20 -1.93372085e-19 -9.05402235e-19
   5.52847144e-04 -8.45971617e-06 -7.04013080e-19]
 [-1.32721528e-20  4.44527460e-21  5.38668155e-21  2.04544456e-19
   8.45971617e-06  5.52847144e-04  1.04041849e-19]
 [-2.60636670e-05 -2.90750246e-05  1.92854913e-04 -1.26016070e-04
  -2.46495168e-19  5.08792645e-20 -1.64871398e-04]]


In [129]:
print(dVf/emeshf,'\n')
print(U_0f)

[ 6.09412400e-21  1.64229498e-20 -1.93372085e-19 -9.05402235e-19
  5.52847144e-04 -8.45971617e-06 -7.04013080e-19 -1.32721528e-20
  4.44527460e-21  5.38668155e-21  2.04544456e-19  8.45971617e-06
  5.52847144e-04  1.04041849e-19 -2.60636670e-05 -2.90750246e-05
  1.92854913e-04 -1.26016070e-04 -2.46495168e-19  5.08792645e-20
 -1.64871398e-04] 

[ 6.09412400e-21  1.64229498e-20 -1.93372085e-19 -9.05402235e-19
  5.52847144e-04 -8.45971617e-06 -7.04013080e-19 -1.32721528e-20
  4.44527460e-21  5.38668155e-21  2.04544456e-19  8.45971617e-06
  5.52847144e-04  1.04041849e-19 -2.60636670e-05 -2.90750246e-05
  1.92854913e-04 -1.26016070e-04 -2.46495168e-19  5.08792645e-20
 -1.64871398e-04]


In [130]:
def emul(_u):
    return _u/emeshf

In [131]:
emul(dVf)

array([ 6.09412400e-21,  1.64229498e-20, -1.93372085e-19, -9.05402235e-19,
        5.52847144e-04, -8.45971617e-06, -7.04013080e-19, -1.32721528e-20,
        4.44527460e-21,  5.38668155e-21,  2.04544456e-19,  8.45971617e-06,
        5.52847144e-04,  1.04041849e-19, -2.60636670e-05, -2.90750246e-05,
        1.92854913e-04, -1.26016070e-04, -2.46495168e-19,  5.08792645e-20,
       -1.64871398e-04])

Now the inverse

In [132]:
print(dVf,'\n')
print(U_0f*emeshf)

[-1.26793162e-19 -1.87993046e-19  3.60980919e-19  9.56057883e-19
 -5.21887952e-04  7.98597586e-06  5.60669325e-19  2.76137837e-19
 -5.08849338e-20 -1.00556875e-20 -2.15988356e-19 -7.98597586e-06
 -5.21887952e-04 -8.28579393e-20  5.63855928e-04  3.56894680e-04
 -5.19696475e-04  2.37405875e-04  4.36785880e-19 -9.01573221e-20
  2.67813078e-04] 

[-1.26793162e-19 -1.87993046e-19  3.60980919e-19  9.56057883e-19
 -5.21887952e-04  7.98597586e-06  5.60669325e-19  2.76137837e-19
 -5.08849338e-20 -1.00556875e-20 -2.15988356e-19 -7.98597586e-06
 -5.21887952e-04 -8.28579393e-20  5.63855928e-04  3.56894680e-04
 -5.19696475e-04  2.37405875e-04  4.36785880e-19 -9.01573221e-20
  2.67813078e-04]


In [133]:
def emul2(_u):
    return _u*emeshf #from U gives dV

In [134]:
print(dVf,'\n')
print(emul2(U_0f))

[-1.26793162e-19 -1.87993046e-19  3.60980919e-19  9.56057883e-19
 -5.21887952e-04  7.98597586e-06  5.60669325e-19  2.76137837e-19
 -5.08849338e-20 -1.00556875e-20 -2.15988356e-19 -7.98597586e-06
 -5.21887952e-04 -8.28579393e-20  5.63855928e-04  3.56894680e-04
 -5.19696475e-04  2.37405875e-04  4.36785880e-19 -9.01573221e-20
  2.67813078e-04] 

[-1.26793162e-19 -1.87993046e-19  3.60980919e-19  9.56057883e-19
 -5.21887952e-04  7.98597586e-06  5.60669325e-19  2.76137837e-19
 -5.08849338e-20 -1.00556875e-20 -2.15988356e-19 -7.98597586e-06
 -5.21887952e-04 -8.28579393e-20  5.63855928e-04  3.56894680e-04
 -5.19696475e-04  2.37405875e-04  4.36785880e-19 -9.01573221e-20
  2.67813078e-04]


In [135]:
def emul3(_u):
    return _u*emeshf-_u #from U gives dV

In [136]:
uk=krylov(emul3,dVf)

In [137]:
print(U_0f)
print(uk)
np.allclose(uk,U_0f)

[ 6.09412400e-21  1.64229498e-20 -1.93372085e-19 -9.05402235e-19
  5.52847144e-04 -8.45971617e-06 -7.04013080e-19 -1.32721528e-20
  4.44527460e-21  5.38668155e-21  2.04544456e-19  8.45971617e-06
  5.52847144e-04  1.04041849e-19 -2.60636670e-05 -2.90750246e-05
  1.92854913e-04 -1.26016070e-04 -2.46495168e-19  5.08792645e-20
 -1.64871398e-04]
[ 2.72464657e-18 -5.62164436e-19 -1.93310908e-19 -9.14132761e-19
  5.52847144e-04 -8.45971617e-06 -6.83459859e-19 -5.93390053e-18
 -1.52163608e-19  5.38497736e-21  2.06516818e-19  8.45971617e-06
  5.52847144e-04  1.01004412e-19 -2.60636670e-05 -2.90750246e-05
  1.92854913e-04 -1.26016070e-04 -2.46181125e-19  5.08144425e-20
 -1.64871398e-04]


True

now add the electronic repulsion response other part !!!!

In [138]:
def gmul(_u):
    return np.einsum('ijkl,kl->ij',G_cphfvovo,_u.reshape(nvir,nocc)).flatten()

In [139]:
def amul(_u):
    return emul3(_u)-gmul(_u)

In [140]:
gmul(uk)

array([-1.76818447e-18,  4.06614542e-19, -1.47551421e-19, -5.94309241e-19,
        3.42455672e-04, -5.24028715e-06, -3.68744799e-19,  3.67475149e-18,
        1.16306379e-19, -1.72650045e-19, -4.56940262e-20,  5.24028715e-06,
        3.42455672e-04,  5.36797317e-20, -3.15105216e-05, -2.58489606e-05,
        2.46600960e-04, -1.77547344e-04, -3.53856764e-19, -4.91269305e-20,
       -2.53085062e-04])

In [141]:
(amul(uk)+uk),dVvo

(array([-5.49202837e-17,  6.02846601e-18,  5.08418137e-19,  1.55958611e-18,
        -8.64343624e-04,  1.32262630e-05,  9.13045734e-19,  1.19784832e-16,
         1.62550618e-18,  1.62597539e-19, -1.72377041e-19, -1.32262630e-05,
        -8.64343624e-04, -1.34118685e-19,  5.95366450e-04,  3.82743640e-04,
        -7.66297436e-04,  4.14953219e-04,  7.90086164e-19, -4.09155281e-20,
         5.20898139e-04]),
 array([[-1.26793162e-19, -1.87993046e-19,  3.60980919e-19,
          9.56057883e-19, -5.21887952e-04,  7.98597586e-06,
          5.60669325e-19],
        [ 2.76137837e-19, -5.08849338e-20, -1.00556875e-20,
         -2.15988356e-19, -7.98597586e-06, -5.21887952e-04,
         -8.28579393e-20],
        [ 5.63855928e-04,  3.56894680e-04, -5.19696475e-04,
          2.37405875e-04,  4.36785880e-19, -9.01573221e-20,
          2.67813078e-04]]))

In [142]:
uk=krylov(amul,dVf)

In [143]:
print(uk)
print(U_0f)

[ 4.48937412e-18 -9.05562410e-19 -1.21695183e-19 -4.80771798e-19
  3.36959549e-04 -5.15618498e-06 -4.48430231e-19 -6.07527315e-18
 -3.31178552e-19 -1.84653600e-20  1.20387196e-19  5.15618498e-06
  3.36959549e-04  5.81862480e-20 -2.55809124e-05 -2.86400691e-05
  1.42063580e-04 -6.58925086e-05 -8.72818632e-20 -1.08090017e-20
 -6.05167367e-05]
[ 6.09412400e-21  1.64229498e-20 -1.93372085e-19 -9.05402235e-19
  5.52847144e-04 -8.45971617e-06 -7.04013080e-19 -1.32721528e-20
  4.44527460e-21  5.38668155e-21  2.04544456e-19  8.45971617e-06
  5.52847144e-04  1.04041849e-19 -2.60636670e-05 -2.90750246e-05
  1.92854913e-04 -1.26016070e-04 -2.46495168e-19  5.08792645e-20
 -1.64871398e-04]


In [144]:
test(U_0f)

0.001789276910303951
0.002966157574886327
0.0012701585584829094


In [145]:
test(uk)

0.001789276910303951
0.0017892788637832555
3.8623960143578005e-09
